In [1]:
import datetime
from qmt_data import QmtData
import os
import pandas as pd
today = datetime.datetime.now().strftime("%Y%m%d")
start_time = "20241101"
end_time = today
qmt_data = QmtData()
ticker_df = qmt_data.get_tickers()
stock_list = ticker_df["ticker"].tolist()
stock_list.sort()
today = datetime.datetime.now().strftime("%Y%m%d")
start_time = "20241101"
qmt_data = QmtData()
ticker_df = qmt_data.get_tickers()
stock_list = ticker_df["ticker"].tolist()
stock_list.sort()
today = datetime.datetime.now().strftime("%Y%m%d")
start_time = "20241101"
end_time = today
qmt_data.remove_qmt_datadir()

qmt_data.download_adjust_factor(
    stock_list=stock_list, start_time=start_time, end_time=end_time
)
for period in ["1m", "5m", "1d"]:
    qmt_data.download_data(
        start_time=start_time,
        end_time=end_time,
        period=period,
        stock_list=stock_list,
    )
    # record = []
    # for stock in tqdm(stock_list):
    #     record.append(
    #         qmt_data.write_to_ftr(
    #             start_time=start_time,
    #             end_time=end_time,
    #             period=period,
    #             stock_code=stock,
    #         )
    #     )
    record = qmt_data.write_to_ftr_parallel(
        stock_list=stock_list,
        start_time=start_time,
        end_time=end_time,
        period=period,
    )
    record_df = pd.DataFrame(record)
    os.makedirs(
        "D:/qmt_datadir/logging/",
        exist_ok=True,
    )
    record_df.to_excel(
        f"D:/qmt_datadir/logging/{end_time}_record_{period}.xlsx",
        index=False,
        engin可以靠8677 任7i6767e="openpyxl",
    )


删除 D:/国金证券QMT交易端/userdata_mini/datadir\SH成功
删除 D:/国金证券QMT交易端/userdata_mini/datadir\SZ成功


  0%|          | 5/7913 [00:00<06:18, 20.87it/s]

Downloaded data for 000001.SH
Downloaded data for 000001.SZ
Downloaded data for 000002.SH
Downloaded data for 000002.SZ
Downloaded data for 000003.SH


  0%|          | 8/7913 [00:00<05:36, 23.49it/s]

Downloaded data for 000004.SH
Downloaded data for 000004.SZ
Downloaded data for 000005.SH


...

100%|█████████▉| 7911/7913 [04:09<00:00, 22.11it/s]

Downloaded data for 980032.SZ
Downloaded data for 980068.SZ
Downloaded data for 988006.SZ
Downloaded data for 988007.SZ
Downloaded data for 988106.SZ


100%|██████████| 7913/7913 [04:09<00:00, 31.73it/s]


Downloaded data for 988107.SZ
Downloaded data for 988201.SZ


100%|██████████| 7913/7913 [00:29<00:00, 271.01it/s]
